In [2]:
import pandas as pd

In [8]:
rental, booking = pd.read_csv(f"..\data\{filename}.csv") for filename in [
    "outdoorsy_rental_data", "outdoorsy_booking_history"
]

In [11]:
rental.head()

,rental_id,listing_state,vehicle_type,is_motorized,vehicle_make,vehicle_model,vehicle_year,vehicle_length,cancellation_policy,sleeps,is_festival_friendly,is_tailgate_friendly,is_pet_friendly,has_toilet,has_generator,has_inside_shower,has_refrigerator,has_wifi
0,206969964720,CA,trailer,False,Airstream,Safari,2008,20 - 25,flexible,4,False,True,True,True,False,True,True,False
1,374663654160,CA,c,True,Forest River,Sunseeker,2012,30 - 35,custom,9,True,True,False,True,True,True,True,False
2,436102091040,CA,c,True,forest river,Sunseeker,2011,30 - 35,strict,8,True,True,True,True,True,True,True,False
3,494831819280,MI,b,True,Leisure Travel,Serenity,2015,25 - 30,strict,4,False,False,True,True,True,True,True,False
4,645780764400,KY,a,True,Thor Motor Coach,Miramar,2016,35 - 40,moderate,10,True,True,False,True,True,True,True,False


In [12]:
booking.head()

,rental_id,booked_price_per_day,month_booked
0,25313374733280,$222.31,7
1,13557825481920,$146.93,11
2,18214711249680,$183.85,9
3,20892146617200,$271.92,7
4,25286903262480,$127.84,6


In [62]:
df = pd.merge(rental, booking, on="rental_id")

In [63]:
df["vehicle_type"].value_counts()

trailer            1299
c                  1054
camper-van          543
b                   392
a                   316
fifth-wheel          84
truck-camper         62
toy-hauler           52
folding-trailer      47
Name: vehicle_type, dtype: int64

In [44]:
# need to combine numerical and string representations
df["month_booked"].value_counts()

6            725
7            671
8            528
9            474
10           472
11           364
12           328
5            236
July          10
September      9
June           8
August         6
December       6
October        6
May            3
November       3
Name: month_booked, dtype: int64

In [66]:
# because of string values, entire column is encoded as object
month_dict = {
    "January": "1",
    "February": "2",
    "March": "3",
    "April": "4",
    "May": "5",
    "June": "6",
    "July": "7",
    "August": "8",
    "September": "9",
    "October": "10",
    "November": "11",
    "December": "12"
}
df = df.assign(month_booked_num=df["month_booked"].replace(month_dict).astype(int))

In [67]:
df["month_booked_num"].value_counts()

6     733
7     681
8     534
9     483
10    478
11    367
12    334
5     239
Name: month_booked_num, dtype: int64

,rental_id,is_motorized,vehicle_year,sleeps,is_festival_friendly,is_tailgate_friendly,is_pet_friendly,has_toilet,has_generator,has_inside_shower,has_refrigerator,has_wifi
month_booked_num,,,,,,,,,,,,
5,2.152349e+13,1.0,2014.764706,6.397059,0.279412,0.617647,0.647059,1.000000,0.926471,1.000000,1.000000,0.147059
6,2.009252e+13,1.0,2014.310160,6.363636,0.213904,0.497326,0.614973,0.962567,0.919786,0.957219,0.967914,0.101604
7,2.116205e+13,1.0,2014.676923,6.451282,0.251282,0.405128,0.564103,0.958974,0.882051,0.948718,0.958974,0.092308
8,2.094310e+13,1.0,2015.042683,6.548780,0.231707,0.530488,0.609756,0.981707,0.939024,0.975610,0.981707,0.030488
9,2.264639e+13,1.0,2015.664000,6.248000,0.224000,0.384000,0.624000,0.992000,0.904000,0.976000,0.992000,0.112000
10,2.153714e+13,1.0,2015.150794,6.325397,0.285714,0.507937,0.603175,0.992063,0.968254,0.992063,0.992063,0.103175
11,2.222886e+13,1.0,2015.980583,6.349515,0.281553,0.533981,0.669903,0.980583,0.970874,0.980583,0.980583,0.097087
12,2.517108e+13,1.0,2015.348837,6.476744,0.162791,0.395349,0.686047,0.965116,0.941860,0.965116,0.953488,0.093023


In [88]:
a = df["booked_price_per_day"].str.replace("[$,]", "", regex=True)

In [106]:
mask = df["booked_price_per_day"] != "#DIV/0!"
df_mask = df[mask]
df_mask = df_mask.assign(
    booked_price_per_day_num=df_mask["booked_price_per_day"].str.replace("[$,]", "", regex=True).astype(float)
)

In [108]:
mask = df_mask["vehicle_type"] == "c"
df_mask[mask].groupby(
    "month_booked_num"
).mean()["booked_price_per_day_num"]

month_booked_num
5     212.670882
6     212.614492
7     233.220361
8     240.465793
9     245.674480
10    242.999286
11    234.897476
12    242.132326
Name: booked_price_per_day_num, dtype: float64